# IDEA

1. fp, sp 인코딩
    - fp, sp가 직접적으로 나오는 full-name, 약자 모두 인코딩
    - 대명사 => X
    - 대문자 => 소문자 X
    - fp: FisrtParty, sp: SecondParty  
        => bert base tokenizer에서 fp, sp 또는 그냥 이름들이 OOV 처리를 받는지 확인  
        
===== Augmentation =====  
1. fp, sp -> sp, fp 바꿔서 label 반전
2. 문장내 fp, sp를 랜덤하게 마스킹
3. 문장내 fp, sp를 랜덤하게 remove ???

===== Hyphothesis =====  
1. fp, sp를 다른 이름으로 바꿈 => label: 2 : padon
2. facts, Q1: who is fp?, Q2: who is sp?, Q3: who won? (fp, sp)


In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

2023-06-29 13:16:37.880736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 13:16:37.941899: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 13:16:37.943893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bluesun/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu:/h

In [2]:
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')

In [3]:
column_rename = {'first_party': 'fp',
                 'second_party': 'sp',
                 'first_party_winner': 'label'}

train_df.rename(columns=column_rename, inplace=True)
test_df.rename(columns=column_rename, inplace=True)

In [4]:
train_df

,ID,fp,sp,facts,label
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [5]:
train_df['label'].value_counts()

1    1649
0     829
Name: label, dtype: int64

In [6]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
train_df, _ = ros.fit_resample(train_df, train_df['label'])

In [7]:
train_df['label'].value_counts()

1    1649
0    1649
Name: label, dtype: int64

In [8]:
aug_df = pd.DataFrame({'ID': train_df['ID'],
                       'fp': train_df['sp'],
                       'sp': train_df['fp'],
                       'facts': train_df['facts'],
                       'label': 1-train_df['label']})

aug_df

,ID,fp,sp,facts,label
0,TRAIN_0000,Herman A. Thompson,Phil A. St. Amant,"On June 27, 1962, Phil St. Amant, a candidate ...",0
1,TRAIN_0001,Lawrence Owens,Stephen Duncan,Ramon Nelson was riding his bike when he suffe...,1
2,TRAIN_0002,"Tony Patterson, Warden, et al.",Billy Joe Magwood,An Alabama state court convicted Billy Joe Mag...,0
3,TRAIN_0003,Walker,Linkletter,Victor Linkletter was convicted in state court...,1
4,TRAIN_0004,Alabama,William Earl Fikes,"On April 24, 1953 in Selma, Alabama, an intrud...",0
...,...,...,...,...,...
3293,TRAIN_2248,Estelle,Rummel,After being convicted of three felonies over a...,1
3294,TRAIN_1654,"District Director, Immigration and Naturalizat...",Kalman J. Berenyi,Kalman Berenyi applied for naturalization as a...,1
3295,TRAIN_1617,United States,Dow Chemical Company,Dow Chemical Company denied the Environmental ...,1
3296,TRAIN_0743,Chakrabarty,Diamond,After genetically engineering a bacterium capa...,1


In [9]:
train_df = pd.concat([train_df, aug_df], ignore_index=True)
train_df

,ID,fp,sp,facts,label
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
6591,TRAIN_2248,Estelle,Rummel,After being convicted of three felonies over a...,1
6592,TRAIN_1654,"District Director, Immigration and Naturalizat...",Kalman J. Berenyi,Kalman Berenyi applied for naturalization as a...,1
6593,TRAIN_1617,United States,Dow Chemical Company,Dow Chemical Company denied the Environmental ...,1
6594,TRAIN_0743,Chakrabarty,Diamond,After genetically engineering a bacterium capa...,1


In [10]:
train_df.isnull().sum()

ID       0
fp       0
sp       0
facts    0
label    0
dtype: int64

In [11]:
# type casting
# train_df['label'] = train_df['label'].astype(str)
# train_df['label'] = train_df['label'].astype(str)

In [11]:
# train_data = Dataset.from_pandas(pd.read_csv('./Data/train.csv'))
# test_data = Dataset.from_pandas(pd.read_csv('./Data/test.csv'))

train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [12]:
train_data.features

{'ID': Value(dtype='string', id=None),
 'fp': Value(dtype='string', id=None),
 'sp': Value(dtype='string', id=None),
 'facts': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [13]:
import datasets

label = datasets.ClassLabel(num_classes=2, names=[0, 1])
train_data.features['label'] = label

In [14]:
train_data['label'][:10]

[1, 0, 1, 0, 1, 1, 1, 1, 1, 1]

In [15]:
train_data.features

{'ID': Value(dtype='string', id=None),
 'fp': Value(dtype='string', id=None),
 'sp': Value(dtype='string', id=None),
 'facts': Value(dtype='string', id=None),
 'label': ClassLabel(names=[0, 1], id=None)}

In [16]:
train_data

Dataset({
    features: ['ID', 'fp', 'sp', 'facts', 'label'],
    num_rows: 6596
})

In [17]:
train_data['label'][:10]

[1, 0, 1, 0, 1, 1, 1, 1, 1, 1]

In [18]:
train_data = train_data.train_test_split(test_size=0.2, shuffle=True, seed=42, stratify_by_column='label')
dataset = DatasetDict({'train': train_data['train'], 'validation': train_data['test'], 'test': test_data})
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'fp', 'sp', 'facts', 'label'],
        num_rows: 5276
    })
    validation: Dataset({
        features: ['ID', 'fp', 'sp', 'facts', 'label'],
        num_rows: 1320
    })
    test: Dataset({
        features: ['ID', 'fp', 'sp', 'facts'],
        num_rows: 1240
    })
})

In [19]:
import numpy as np

np.mean(train_data['train']['label']), np.mean(train_data['test']['label'])

(0.5, 0.5)

In [8]:
# pretrained_model = "distilbert-base-uncased"
pretrained_model = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [9]:
def preprocess(examples):
    examples['facts_with_party'] = [f'First: {first}.\n Second: {second}.\nFacts: {fact}' 
                                    for first, second, fact in zip(examples['first_party'], examples['second_party'], examples['facts'])]
    return tokenizer(examples['facts_with_party'], padding='max_length', truncation=True, max_length=512)

dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/1982 [00:00<?, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/1240 [00:00<?, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'first_party', 'second_party', 'facts', 'label', 'facts_with_party', 'input_ids', 'attention_mask'],
        num_rows: 1982
    })
    validation: Dataset({
        features: ['ID', 'first_party', 'second_party', 'facts', 'label', 'facts_with_party', 'input_ids', 'attention_mask'],
        num_rows: 496
    })
    test: Dataset({
        features: ['ID', 'first_party', 'second_party', 'facts', 'facts_with_party', 'input_ids', 'attention_mask'],
        num_rows: 1240
    })
})

In [11]:
import evaluate
import numpy as np

accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return accuracy.compute(predictions=preds, references=labels)

In [12]:
from transformers import DistilBertModel

model = DistilBertModel.from_pretrained(pretrained_model, num_labels=2)

You are using a model of type xlm-roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing DistilBertModel: ['roberta.encoder.layer.4.attention.self.key.weight', 'roberta.encoder.layer.7.attention.self.value.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.9.attention.self.value.weight', 'lm_head.dense.bias', 'roberta.encoder.layer.6.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.intermediate.dense.weight', 'lm_head.decoder.weight', 'roberta.encoder.layer.5.attention.output.LayerNorm.weight', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.3.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.self.key.weight', 'roberta.encoder.layer.5.attention.self.value.weight', 'roberta.encoder.layer.8.attention.self.key.weight', 'roberta.encoder.layer

In [48]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [49]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: second_party, first_party, ID, facts, facts_with_party. If second_party, first_party, ID, facts, facts_with_party are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/bluesun/anaconda3/envs/nlp/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1982
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 620
  Number of trainable parameters = 66955010


  0%|          | 0/620 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: second_party, first_party, ID, facts, facts_with_party. If second_party, first_party, ID, facts, facts_with_party are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 496
  Batch size = 16


  0%|          | 0/31 [00:00<?, ?it/s]

Saving model checkpoint to ./results/checkpoint-124
Configuration saved in ./results/checkpoint-124/config.json


...

Saving model checkpoint to ./results/checkpoint-620
Configuration saved in ./results/checkpoint-620/config.json


{'eval_loss': 1.1904042959213257, 'eval_accuracy': 0.5725806451612904, 'eval_runtime': 2.2313, 'eval_samples_per_second': 222.294, 'eval_steps_per_second': 13.893, 'epoch': 5.0}


Model weights saved in ./results/checkpoint-620/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-248 (score: 0.6471774193548387).


{'train_runtime': 145.2678, 'train_samples_per_second': 68.219, 'train_steps_per_second': 4.268, 'train_loss': 0.48051954084827053, 'epoch': 5.0}


TrainOutput(global_step=620, training_loss=0.48051954084827053, metrics={'train_runtime': 145.2678, 'train_samples_per_second': 68.219, 'train_steps_per_second': 4.268, 'train_loss': 0.48051954084827053, 'epoch': 5.0})

In [50]:
import torch as th

with th.no_grad():
    logits = trainer.predict(dataset['validation'])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: second_party, first_party, ID, facts, facts_with_party. If second_party, first_party, ID, facts, facts_with_party are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 496
  Batch size = 16


  0%|          | 0/31 [00:00<?, ?it/s]

In [51]:
preds = np.argmax(logits.predictions, axis=1)

In [52]:
logits.label_ids

array([1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,

In [53]:
np.array(dataset['validation']['label']) - logits.label_ids

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [54]:
logits.metrics

{'test_loss': 0.6946614384651184,
 'test_accuracy': 0.6471774193548387,
 'test_runtime': 2.2437,
 'test_samples_per_second': 221.067,
 'test_steps_per_second': 13.817}

In [55]:
with th.no_grad():
    logits = trainer.predict(dataset['test'])


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: second_party, first_party, ID, facts, facts_with_party. If second_party, first_party, ID, facts, facts_with_party are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1240
  Batch size = 16


  0%|          | 0/78 [00:00<?, ?it/s]

In [56]:
logits

PredictionOutput(predictions=array([[ 0.03335458,  0.0423288 ],
       [-0.23478237,  0.32663944],
       [ 0.16238324, -0.07501629],
       ...,
       [-0.4196741 ,  0.53455144],
       [-0.13286527,  0.15887444],
       [-0.8414401 ,  0.93614745]], dtype=float32), label_ids=None, metrics={'test_runtime': 5.5602, 'test_samples_per_second': 223.015, 'test_steps_per_second': 14.028})

In [57]:
preds = logits.predictions.argmax(axis=1)

In [58]:
preds.mean()

0.9612903225806452

In [59]:
dataset['test']

Dataset({
    features: ['ID', 'first_party', 'second_party', 'facts', 'facts_with_party', 'input_ids', 'attention_mask'],
    num_rows: 1240
})

In [60]:
submission = pd.DataFrame({'ID': dataset['test']['ID'], 'first_party_winner': preds})
submission

,ID,first_party_winner
0,TEST_0000,1
1,TEST_0001,1
2,TEST_0002,0
3,TEST_0003,1
4,TEST_0004,1
...,...,...
1235,TEST_1235,1
1236,TEST_1236,1
1237,TEST_1237,1
1238,TEST_1238,1


In [61]:
submission.to_csv('submission.csv', index=False, header=True)

In [14]:
import torch as th

def to_input(batch):
    input_ids = th.tensor(batch['input_ids'])
    attention_mask = th.tensor(batch['attention_mask'])
    return {'input_ids': input_ids, 'attention_mask': attention_mask}

input_data = to_input(dataset['test'][0:2])

In [1]:
model(**input_data)['last_hidden_state'].shape

NameError: name 'model' is not defined

In [15]:
train_input_data = to_input(dataset['train'])
test_input_data = to_input(dataset['test'])

In [31]:
train_emb = model(**train_input_data)['last_hidden_state'][:, 0, :].numpy()

: 

: 

In [16]:
test_emb = model(**test_input_data)['last_hidden_state'][:, 0, :].numpy()

: 

: 

AttributeError: 'str' object has no attribute 'size'